In [303]:
import pandas as pd
import numpy as np
import json
import itertools
from apyori import apriori  

In [304]:
df = pd.read_csv("../data/location1_cleaned.csv")
del df['product_sku']
del df['display_description']
del df['location_name']
df_prods = pd.read_csv("../data_engineering/products_clean.csv")
df_prods_idx = df_prods.copy().set_index('id')

def get_name(id):
    return df_prods_idx.loc[id,'name']

def get_cat(id):
    return df_prods_idx.loc[id,'category']

def get_name_cat(id):
    return (df_prods_idx.loc[id,'name'],df_prods_idx.loc[id,'category'])
    
    

In [305]:
#Categories:
cats = list(set(list(df_prods['category'])))
cats

['Dessert', 'Side', 'Alcoholic Beverage', 'Entree', 'Beverage', 'Kids']

In [306]:
df_prods[ df_prods['category'] == "Kids" ] 

,id,name,display_desc,category,img
2,1848,Kids Mac & Cheese,"served with Carrot Sticks, Apple Slices and ch...",Kids,mac-n-cheese.png
3,1851,Smuckers Uncrustables,"served with Carrot Sticks, Apple Slices and ch...",Kids,Smuckers_Uncrustables.jpg
19,2243,Kids Cheeseburger,served with choice of Sides and Beverage,Kids,burger_cheeseburger.jpeg
20,2244,Kids Turkey Sandwich,served with choice of Sides and Beverage,Kids,sandwich_turkey_sandwich.png
30,3020,Kids Chicken Strips,served with choice of Sides and Beverage,Kids,fried_chicken-strips.png
42,3371,Kids BBQ Quesadilla,"served with Steak Fries, Roasted Corn and choi...",Kids,mex_quesadilla.png
51,4751,Kids Hamburger,"served with Carrots, Apple Slices and choice o...",Kids,burger.jpeg


In [307]:
# kids order ids
kids_orders = list(set(list(df[ df['item_category_type'] == 'Kids']['order_id'])))
kids_orders
print(len(kids_orders))

5874


In [308]:
# df frame that has only orders involving a kids order
pd.set_option('display.max_rows', 200)
df_kids = (df.set_index('order_id').loc[ kids_orders ]).reset_index(drop=False)
df_kids.head(200)

,order_id,order_time(UTC),items_id,items_name,item_price,quantity,served_for,item_category_type
0,884739,8/18/18 19:57,2218,Strawberry Lemonade Slushy,5.99,1,Lunch,Beverage
1,884739,8/18/18 19:57,2245,Olaf Chocolate Cupcake,5.99,1,Lunch,Dessert
2,884739,8/18/18 19:57,2245,Olaf Chocolate Cupcake,5.99,1,Lunch,Dessert
3,884739,8/18/18 19:57,2245,Olaf Chocolate Cupcake,5.99,1,Lunch,Dessert
4,884739,8/18/18 19:57,1813,French Fries,3.79,1,Lunch,Side
5,884739,8/18/18 19:57,2238,Chicken Club,10.99,1,Lunch,Entree
6,884739,8/18/18 19:57,2243,Kids Cheeseburger,6.99,1,Lunch,Kids
7,884739,8/18/18 19:57,2716,Chicken Strips,9.49,1,Lunch,Entree
8,884739,8/18/18 19:57,2716,Chicken Strips,9.49,1,Lunch,Entree
9,917508,9/2/18 17:44,2307,1/3 lb Angus Cheeseburger,10.49,1,Lunch,Entree


In [309]:
# NOT USED, dataframe removing entrees and alcoholic beverages
df_kids2 = df_kids[ df_kids['item_category_type'] != 'Entree' ]
df_kids3 = df_kids2[ df_kids2['item_category_type'] != 'Alcoholic Beverage']
del df_kids3['order_time(UTC)']
del df_kids3['item_price']
del df_kids3['quantity']
del df_kids3['served_for']
df_kids3.head(100)

,order_id,items_id,items_name,item_category_type
0,884739,2218,Strawberry Lemonade Slushy,Beverage
1,884739,2245,Olaf Chocolate Cupcake,Dessert
2,884739,2245,Olaf Chocolate Cupcake,Dessert
3,884739,2245,Olaf Chocolate Cupcake,Dessert
4,884739,1813,French Fries,Side
6,884739,2243,Kids Cheeseburger,Kids
11,917508,3020,Kids Chicken Strips,Kids
12,917508,3020,Kids Chicken Strips,Kids
13,917508,2208,Chocolate Mousse,Dessert
14,917508,2208,Chocolate Mousse,Dessert


In [310]:
# get dataframe of kids orders that do not include
# an entree or alcohol
df_entrees_kids_ids = list(set(list(df_kids[ (df_kids['item_category_type'] == 'Entree') | 
                                             (df_kids['item_category_type'] == 'Alcoholic Beverage')]['order_id']
                                   )))
exc_kids_ids = list(set(kids_orders)^set(df_entrees_kids_ids))

df_only_kids = (df.set_index('order_id').loc[ exc_kids_ids ]).reset_index(drop=False)
# remove coffee and tea orders
df_only_kids = df_only_kids[ (df_only_kids['items_id'] != 3070) &
                             (df_only_kids['items_id'] != 3069) &
                             (df_only_kids['items_id'] != 1854)]


print(len(df_only_kids))

2175


In [311]:
# Here we build the 2D orders array that will be fed into apyori
#
# The idea is the following: for each order, we consider all possible combinations
# of kids,side,beverage,dessert to handle the fact that often more than one of
# these categories is purchased. Based on the data, it's impossible to discern
# what belongs together, so we need to account for all possiblities. 
# Of course, this will reduce the predictive power of the outcome.
#

kids_orders = []

# loop over all orders that fit our criteria
for i in range(len(exc_kids_ids)):
    
    # get all items for each order
    dd = (df_only_kids.loc[ df_only_kids['order_id'] == exc_kids_ids[i]]).copy()
    # drop duplicate items
    dd.drop_duplicates(subset='items_id',inplace=True)

    kids = list( dd.loc[ dd['item_category_type'] == 'Kids', 'items_id']  )
    nkids = len(kids)    
    dessert = list(dd.loc[ dd['item_category_type'] == 'Dessert', 'items_id'])
    ndessert = len(dessert)
    bev = list(dd.loc[ dd['item_category_type'] == 'Beverage','items_id'])
    nbev = len(bev)
    side = list(dd.loc[ dd['item_category_type'] == 'Side', 'items_id'])
    nside = len(side)
    
    #if i == 857:
    print(f"{i} nk: {nkids} ns: {nside} nb: {nbev} nd: {ndessert}")

    #if i == 237:
    #    print(dd)
    
    # loop over all kids stuff
    for j in range(nkids):
        this_order = []
        if nside == 0 and nbev == 0 and ndessert == 0:
            this_order.append( [ kids[j] ] )
            continue

        if nside > 0:
            if nbev == 0 and ndessert == 0:
                xx = list(itertools.combinations(side,1))
            if nbev > 0 and ndessert == 0:
                xx = list(itertools.product(side,bev))
            if nbev == 0 and ndessert > 0:
                xx = list(itertools.product(side,dessert))
            if nbev > 0 and ndessert > 0:
                xx = list(itertools.product(side,dessert,bev))
        else:
            if nbev == 0 and ndessert == 0:
               assert(False) # this should never be reached
            if nbev > 0 and ndessert == 0:
                xx = list(itertools.combinations(bev,1))
            if nbev == 0 and ndessert > 0:
                xx = list(itertools.combinations(dessert,1))
            if nbev > 0 and ndessert > 0:
                xx = list(itertools.product(bev,dessert))
            
        for k in range(len(xx)):
            temp_order = []
            temp_order.append(kids[j])
            for l in range(len(xx[k])):
                temp_order.append(xx[k][l])
            this_order.append(temp_order)

    kids_orders += this_order
    

0 nk: 3 ns: 0 nb: 0 nd: 0
1 nk: 1 ns: 0 nb: 1 nd: 0
2 nk: 1 ns: 0 nb: 0 nd: 0
3 nk: 1 ns: 0 nb: 1 nd: 1
4 nk: 1 ns: 0 nb: 0 nd: 0
5 nk: 1 ns: 0 nb: 0 nd: 0
6 nk: 1 ns: 0 nb: 0 nd: 0
7 nk: 1 ns: 0 nb: 0 nd: 0
8 nk: 1 ns: 0 nb: 0 nd: 0
9 nk: 1 ns: 0 nb: 1 nd: 1
10 nk: 1 ns: 0 nb: 0 nd: 0
11 nk: 1 ns: 0 nb: 0 nd: 0
12 nk: 1 ns: 0 nb: 0 nd: 0
13 nk: 2 ns: 0 nb: 0 nd: 0
14 nk: 1 ns: 0 nb: 0 nd: 0
15 nk: 1 ns: 0 nb: 0 nd: 0
16 nk: 3 ns: 0 nb: 0 nd: 0
17 nk: 1 ns: 0 nb: 0 nd: 0
18 nk: 2 ns: 0 nb: 0 nd: 0
19 nk: 1 ns: 0 nb: 0 nd: 0
20 nk: 1 ns: 0 nb: 0 nd: 0
21 nk: 2 ns: 0 nb: 0 nd: 0
22 nk: 1 ns: 0 nb: 0 nd: 0
23 nk: 3 ns: 0 nb: 1 nd: 0
24 nk: 1 ns: 0 nb: 0 nd: 0
25 nk: 2 ns: 0 nb: 0 nd: 1
26 nk: 1 ns: 0 nb: 0 nd: 0
27 nk: 1 ns: 0 nb: 0 nd: 0
28 nk: 1 ns: 0 nb: 0 nd: 0
29 nk: 1 ns: 0 nb: 0 nd: 0
30 nk: 1 ns: 0 nb: 0 nd: 1
31 nk: 2 ns: 0 nb: 0 nd: 0
32 nk: 1 ns: 0 nb: 0 nd: 0
33 nk: 2 ns: 0 nb: 0 nd: 0
34 nk: 1 ns: 0 nb: 0 nd: 0
35 nk: 1 ns: 0 nb: 0 nd: 0
36 nk: 3 ns: 0 nb: 0 nd: 0
37 nk: 1 ns

321 nk: 2 ns: 0 nb: 0 nd: 0
322 nk: 1 ns: 0 nb: 0 nd: 0
323 nk: 1 ns: 0 nb: 0 nd: 0
324 nk: 1 ns: 0 nb: 0 nd: 1
325 nk: 1 ns: 0 nb: 0 nd: 0
326 nk: 1 ns: 0 nb: 0 nd: 0
327 nk: 1 ns: 0 nb: 1 nd: 0
328 nk: 1 ns: 0 nb: 0 nd: 0
329 nk: 1 ns: 0 nb: 0 nd: 0
330 nk: 1 ns: 0 nb: 0 nd: 0
331 nk: 2 ns: 0 nb: 0 nd: 0
332 nk: 1 ns: 1 nb: 0 nd: 0
333 nk: 1 ns: 0 nb: 0 nd: 0
334 nk: 1 ns: 0 nb: 0 nd: 0
335 nk: 1 ns: 0 nb: 0 nd: 0
336 nk: 1 ns: 0 nb: 1 nd: 0
337 nk: 1 ns: 1 nb: 1 nd: 0
338 nk: 1 ns: 0 nb: 0 nd: 0
339 nk: 2 ns: 0 nb: 0 nd: 0
340 nk: 2 ns: 1 nb: 0 nd: 0
341 nk: 1 ns: 0 nb: 0 nd: 0
342 nk: 3 ns: 0 nb: 0 nd: 0
343 nk: 1 ns: 0 nb: 0 nd: 0
344 nk: 1 ns: 0 nb: 0 nd: 0
345 nk: 3 ns: 0 nb: 0 nd: 0
346 nk: 1 ns: 0 nb: 0 nd: 0
347 nk: 1 ns: 0 nb: 0 nd: 0
348 nk: 1 ns: 0 nb: 0 nd: 0
349 nk: 1 ns: 0 nb: 0 nd: 0
350 nk: 1 ns: 0 nb: 0 nd: 0
351 nk: 1 ns: 0 nb: 0 nd: 0
352 nk: 1 ns: 0 nb: 0 nd: 0
353 nk: 3 ns: 0 nb: 1 nd: 0
354 nk: 1 ns: 0 nb: 0 nd: 0
355 nk: 1 ns: 0 nb: 0 nd: 0
356 nk: 1 ns: 0 nb: 

652 nk: 1 ns: 0 nb: 1 nd: 0
653 nk: 1 ns: 0 nb: 0 nd: 0
654 nk: 1 ns: 0 nb: 0 nd: 0
655 nk: 1 ns: 0 nb: 0 nd: 1
656 nk: 1 ns: 0 nb: 0 nd: 0
657 nk: 1 ns: 0 nb: 0 nd: 0
658 nk: 2 ns: 0 nb: 0 nd: 0
659 nk: 1 ns: 0 nb: 0 nd: 0
660 nk: 1 ns: 0 nb: 0 nd: 0
661 nk: 1 ns: 0 nb: 0 nd: 0
662 nk: 1 ns: 0 nb: 0 nd: 0
663 nk: 1 ns: 0 nb: 0 nd: 0
664 nk: 1 ns: 1 nb: 1 nd: 0
665 nk: 2 ns: 0 nb: 1 nd: 0
666 nk: 2 ns: 0 nb: 0 nd: 0
667 nk: 1 ns: 0 nb: 0 nd: 0
668 nk: 1 ns: 0 nb: 1 nd: 0
669 nk: 1 ns: 0 nb: 0 nd: 0
670 nk: 1 ns: 0 nb: 0 nd: 0
671 nk: 1 ns: 1 nb: 2 nd: 0
672 nk: 2 ns: 0 nb: 0 nd: 0
673 nk: 1 ns: 0 nb: 0 nd: 0
674 nk: 1 ns: 0 nb: 0 nd: 0
675 nk: 1 ns: 0 nb: 0 nd: 0
676 nk: 2 ns: 0 nb: 0 nd: 0
677 nk: 1 ns: 1 nb: 0 nd: 0
678 nk: 1 ns: 0 nb: 0 nd: 0
679 nk: 1 ns: 0 nb: 0 nd: 0
680 nk: 3 ns: 0 nb: 1 nd: 0
681 nk: 1 ns: 0 nb: 0 nd: 0
682 nk: 1 ns: 0 nb: 1 nd: 0
683 nk: 1 ns: 0 nb: 0 nd: 0
684 nk: 1 ns: 0 nb: 0 nd: 0
685 nk: 1 ns: 0 nb: 0 nd: 0
686 nk: 1 ns: 0 nb: 0 nd: 0
687 nk: 1 ns: 0 nb: 

980 nk: 2 ns: 0 nb: 0 nd: 0
981 nk: 2 ns: 0 nb: 0 nd: 0
982 nk: 1 ns: 0 nb: 0 nd: 0
983 nk: 1 ns: 0 nb: 0 nd: 0
984 nk: 2 ns: 0 nb: 0 nd: 0
985 nk: 1 ns: 0 nb: 0 nd: 0
986 nk: 1 ns: 0 nb: 0 nd: 0
987 nk: 1 ns: 0 nb: 1 nd: 0
988 nk: 1 ns: 0 nb: 0 nd: 0
989 nk: 4 ns: 1 nb: 0 nd: 0
990 nk: 1 ns: 0 nb: 1 nd: 0
991 nk: 1 ns: 0 nb: 0 nd: 0
992 nk: 1 ns: 0 nb: 0 nd: 0
993 nk: 1 ns: 1 nb: 0 nd: 0
994 nk: 1 ns: 0 nb: 0 nd: 0
995 nk: 1 ns: 0 nb: 0 nd: 0
996 nk: 1 ns: 0 nb: 0 nd: 0
997 nk: 1 ns: 0 nb: 0 nd: 0
998 nk: 2 ns: 0 nb: 0 nd: 0
999 nk: 1 ns: 0 nb: 0 nd: 0
1000 nk: 1 ns: 0 nb: 0 nd: 0
1001 nk: 1 ns: 0 nb: 2 nd: 0
1002 nk: 1 ns: 0 nb: 0 nd: 0
1003 nk: 1 ns: 0 nb: 0 nd: 0
1004 nk: 1 ns: 1 nb: 0 nd: 0
1005 nk: 1 ns: 0 nb: 0 nd: 0
1006 nk: 1 ns: 0 nb: 0 nd: 0
1007 nk: 3 ns: 0 nb: 0 nd: 0
1008 nk: 1 ns: 0 nb: 0 nd: 0
1009 nk: 1 ns: 0 nb: 0 nd: 0
1010 nk: 2 ns: 0 nb: 0 nd: 0
1011 nk: 1 ns: 1 nb: 0 nd: 0
1012 nk: 2 ns: 1 nb: 1 nd: 0
1013 nk: 2 ns: 0 nb: 0 nd: 0
1014 nk: 1 ns: 1 nb: 0 nd: 0
1015 

In [319]:
# run the apriori algorithm using apyori
# Since we have few data sets, we only consider pairs {X} -> {Y}
# We limit lift to > 1.5. The min_support and min_confidence levels are chosen
# to get at least a few cases with multiple possible combinations. We then
# pick the one with the greatest product of support * confidence * lift.

association_rules = apriori(kids_orders, min_support=0.001, min_confidence=0.01, min_lift=1.5, max_length=2)  
assoc = list(association_rules)  

In [313]:
print(len(assoc)) 

29


In [314]:
# messy
for item in assoc:
    print(item.items)

frozenset({1852, 1813})
frozenset({1858, 1813})
frozenset({2218, 1813})
frozenset({2243, 1813})
frozenset({3411, 1813})
frozenset({1848, 1838})
frozenset({1858, 1838})
frozenset({1848, 2221})
frozenset({1848, 2234})
frozenset({1848, 3289})
frozenset({1851, 1852})
frozenset({1851, 2245})
frozenset({2244, 1852})
frozenset({3289, 1852})
frozenset({3411, 1852})
frozenset({1858, 2221})
frozenset({2234, 1858})
frozenset({1858, 2245})
frozenset({3289, 1858})
frozenset({1858, 3411})
frozenset({2244, 2207})
frozenset({2218, 2243})
frozenset({2218, 3411})
frozenset({3371, 2221})
frozenset({2234, 3371})
frozenset({3289, 2245})
frozenset({3289, 3371})
frozenset({3289, 3411})
frozenset({3408, 3371})


In [315]:
# get products sans entrees and alcohol
df_sa_prods = df_prods.loc[ (df_prods['category'] != 'Entree') & 
                            (df_prods['category'] != 'Alcoholic Beverage'), ['id','name','category']].copy()
# remove coffee and tea
df_sa_prods = df_sa_prods[ df_sa_prods['id'] != 3069 ]
df_sa_prods = df_sa_prods[ df_sa_prods['id'] != 3070 ]
df_sa_prods = df_sa_prods[ df_sa_prods['id'] != 1854 ]

df_sa_prods.sort_values('category')

,id,name,category
12,2218,Strawberry Lemonade Slushy,Beverage
22,2251,smartwater,Beverage
4,1852,Bottled Water,Beverage
6,1855,Chocolate Milk,Beverage
7,1856,Lowfat Milk,Beverage
8,1858,Fountain Beverage,Beverage
46,3411,Toy Story Land Cupcake,Dessert
21,2245,Olaf Chocolate Cupcake,Dessert
48,3893,Peanut Butter Chocolate Cake,Dessert
49,4307,Cookies,Dessert


In [316]:
# build the recommendation dict of dicts for Kids
prodlist = df_sa_prods.T.to_dict().values()

reco_dict = {}
cats = ['Kids','Side','Dessert','Beverage']


for prod in prodlist:
    print(prod)
    md = {}
    id = prod['id']
    md['category'] = prod['category']
    md[prod['category']] = -1
    md['Entree'] = -1

    search_cats = [ x for x in cats if x != prod['category'] ]
    print(search_cats)
    print(id)
    
    dd = [ x for x in assoc if list(x.items)[0] == id or list(x.items)[1] == id  ]
    # for each category, we take the pairing with the highest
    # support, since we have already placed a good limit on lift
    # If there is no pairing, we set that category to -2
    for cat in search_cats:
        
        xl = []
        for x in dd:
            for y in x.ordered_statistics:
                xd = {}
                xid = list(y.items_add)[0]
                if xid != id:
                    xd['id'] = xid
                    xd['support'] = x.support
                    xd['confidence'] = y.confidence
                    xd['lift'] = y.lift
                    if get_cat(xid) == cat:
                        xl.append(xd)    


        
        xs = sorted(xl,key = lambda x: x['support']*x['confidence']*x['lift'],reverse=True)
        if len(xs) > 0:
            md[cat] = xs[0]
        else:
            md[cat] = -2
        print(cat,xs)
        
        print('**')

    print('------------------')
        
    
    
    reco_dict[prod['id']] = md
    
    
    

{'id': 1813, 'name': 'French Fries', 'category': 'Side'}
['Kids', 'Dessert', 'Beverage']
1813
Kids [{'id': 2243, 'support': 0.00945179584120983, 'confidence': 0.3125, 'lift': 1.5236175115207373}]
**
Dessert [{'id': 3411, 'support': 0.003780718336483932, 'confidence': 0.125, 'lift': 3.3910256410256414}]
**
Beverage [{'id': 1858, 'support': 0.00945179584120983, 'confidence': 0.3125, 'lift': 3.936011904761905}, {'id': 1852, 'support': 0.003780718336483932, 'confidence': 0.125, 'lift': 5.510416666666666}, {'id': 2218, 'support': 0.001890359168241966, 'confidence': 0.0625, 'lift': 2.06640625}]
**
------------------
{'id': 1838, 'name': 'Green Beans', 'category': 'Side'}
['Kids', 'Dessert', 'Beverage']
1838
Kids [{'id': 1848, 'support': 0.003780718336483932, 'confidence': 0.4444444444444445, 'lift': 1.851268591426072}]
**
Dessert []
**
Beverage [{'id': 1858, 'support': 0.001890359168241966, 'confidence': 0.22222222222222224, 'lift': 2.7989417989417995}]
**
------------------
{'id': 1848, 'na

In [317]:
print(json.dumps(reco_dict,indent=2))

{
  "1813": {
    "category": "Side",
    "Side": -1,
    "Entree": -1,
    "Kids": {
      "id": 2243,
      "support": 0.00945179584120983,
      "confidence": 0.3125,
      "lift": 1.5236175115207373
    },
    "Dessert": {
      "id": 3411,
      "support": 0.003780718336483932,
      "confidence": 0.125,
      "lift": 3.3910256410256414
    },
    "Beverage": {
      "id": 1858,
      "support": 0.00945179584120983,
      "confidence": 0.3125,
      "lift": 3.936011904761905
    }
  },
  "1838": {
    "category": "Side",
    "Side": -1,
    "Entree": -1,
    "Kids": {
      "id": 1848,
      "support": 0.003780718336483932,
      "confidence": 0.4444444444444445,
      "lift": 1.851268591426072
    },
    "Dessert": -2,
    "Beverage": {
      "id": 1858,
      "support": 0.001890359168241966,
      "confidence": 0.22222222222222224,
      "lift": 2.7989417989417995
    }
  },
  "1848": {
    "category": "Kids",
    "Kids": -1,
    "Entree": -1,
    "Side": {
      "id": 3289,
   

In [318]:
with open("../data/kids_recommendations.json",'w') as f:
    f.write(json.dumps(reco_dict,indent=2))